In [1]:
import pandas as pd

Question 7 : Verification lignes attributs.

In [2]:
#Q7
chemin_fichier_csv = r'C:\Users\ilyes\Desktop\ProjetIa\champignons.csv'

champignons = pd.read_csv(chemin_fichier_csv)

print(f"Forme du DataFrame : {champignons.shape}")

print(champignons.head())

Forme du DataFrame : (1158, 4)
  TYPE         COLOR       SHAPE     SURFACE
0    I          Pale    Polypore      Smooth
1    E         White      Convex      Smooth
2    E  White-Yellow      Convex      Smooth
3    E         Brown  Bellshaped  FlatScales
4    E           NaN      Convex     Fibrous


Question 8 : Commencer par inspecter les données de cette colonne via la méthode value_counts().

In [3]:
#Q8
edible_compteur = champignons['TYPE'].value_counts(dropna=False)

print(edible_compteur)


TYPE
E      515
P      404
I      228
NaN     11
Name: count, dtype: int64


Question 9 : remplacer les valeur "E","I" et "P" par 0,1 et 2.

In [4]:
#Q9
champignons['TYPE'] = champignons['TYPE'].replace({'E': 0, 'I': 1, 'P': 2})

print(champignons.head())

   TYPE         COLOR       SHAPE     SURFACE
0   1.0          Pale    Polypore      Smooth
1   0.0         White      Convex      Smooth
2   0.0  White-Yellow      Convex      Smooth
3   0.0         Brown  Bellshaped  FlatScales
4   0.0           NaN      Convex     Fibrous


Question 10 : remplacer les valeurs manquantes par -1

In [5]:
#Q10

champignons['TYPE'] = champignons['TYPE'].fillna(-1)

new_edible_compteur = champignons['TYPE'].value_counts(dropna=False)

print(new_edible_compteur)

TYPE
 0.0    515
 2.0    404
 1.0    228
-1.0     11
Name: count, dtype: int64


Question 11 : En consultant la documentation, comprendre ce que fait cette expression.

In [6]:
# pd . unique ( champignons [ " Shape " ]. str. split ( " -" ). explode (). dropna ())

# cette expression prend la colonne "Shape" du DataFrame champignons, la divise en sous-chaînes à chaque occurrence du caractère "-", 
# puis la transforme en une série avec des valeurs individuelles. Ensuite, elle élimine les valeurs nulles et renvoie la liste de 
# toutes les valeurs uniques présentes dans cette colonne après cette transformation. Cela permet d'obtenir une liste complète de 
# toutes les formes possibles de champignons.

Question 12 : Ajout d'une nouvelle colonne remplie par des 0 et des 1

Question 13 : Meme chose pour la colonne surface (On factorise au maximum). On supprime la colonne shape et surface

In [7]:
#Q12-13

import warnings
warnings.filterwarnings("ignore")

def ajout_indicateur_colonne(df, nom_colonne):
    valeur_unique = pd.unique(df[nom_colonne].str.split("-").explode().dropna())
    
    for value in valeur_unique:
        df[f"{nom_colonne}_{value}"] = df[nom_colonne].str.contains(value, na=False).astype(int)

ajout_indicateur_colonne(champignons, "SHAPE")

ajout_indicateur_colonne(champignons, "SURFACE")

print(champignons.head())

champignons = champignons.drop(['SHAPE', 'SURFACE'], axis=1)

print(f"Taille du DataFrame après suppression : {champignons.shape}")

   TYPE         COLOR       SHAPE     SURFACE  SHAPE_Polypore  SHAPE_Convex  \
0   1.0          Pale    Polypore      Smooth               1             0   
1   0.0         White      Convex      Smooth               0             1   
2   0.0  White-Yellow      Convex      Smooth               0             1   
3   0.0         Brown  Bellshaped  FlatScales               0             0   
4   0.0           NaN      Convex     Fibrous               0             1   

   SHAPE_Bellshaped  SHAPE_Depressed  SHAPE_CupFungi  SHAPE_CoralFungi  ...  \
0                 0                0               0                 0  ...   
1                 0                0               0                 0  ...   
2                 0                0               0                 0  ...   
3                 1                0               0                 0  ...   
4                 0                0               0                 0  ...   

   SURFACE_Silky  SURFACE_Velvety  SURFACE_AlanRoc

Question 14 : la liste des couleurs individuelles présentes dans notre jeu de donnée.

In [8]:
#Q14

couleurs_individuelles = pd.unique(champignons["COLOR"].str.split("-").explode().dropna())

print(couleurs_individuelles)

print(f"Nombre total de couleurs uniques : {len(couleurs_individuelles)}")

['Pale' 'White' 'Yellow' 'Brown' 'Pink' 'Purple' 'Tan' 'Orange' 'Gray'
 'Red' 'Black' 'Blue' 'Green' 'Violet' 'Lilac']
Nombre total de couleurs uniques : 15


Question 15 : création d'un DataFrame comportant 4 colonnes.

In [9]:
#Q15

colors_list = [
    {"Color": "Pale", "R": 240, "G": 221, "B": 215},
    {"Color": "White", "R": 255, "G": 255, "B": 255},
    {"Color": "Yellow", "R": 255, "G": 255, "B": 0},
    {"Color": "Brown", "R": 165, "G": 42, "B": 42},
    {"Color": "Pink", "R": 255, "G": 192, "B": 203},
    {"Color": "Purple", "R": 128, "G": 0, "B": 128},
    {"Color": "Tan", "R": 210, "G": 180, "B": 140},
    {"Color": "Orange", "R": 255, "G": 165, "B": 0},
    {"Color": "Gray", "R": 128, "G": 128, "B": 128},
    {"Color": "Red", "R": 255, "G": 0, "B": 0},
    {"Color": "Black", "R": 0, "G": 0, "B": 0}, #Dark
    {"Color": "Green", "R": 0, "G": 128, "B": 0},
    {"Color": "Blue", "R": 0, "G": 0, "B": 255},
    {"Color": "Violet", "R": 238, "G": 130, "B": 238},
    {"Color": "Lilac", "R": 200, "G": 162, "B": 200}
]

colors_df = pd.DataFrame(colors_list)

print(colors_df)

     Color    R    G    B
0     Pale  240  221  215
1    White  255  255  255
2   Yellow  255  255    0
3    Brown  165   42   42
4     Pink  255  192  203
5   Purple  128    0  128
6      Tan  210  180  140
7   Orange  255  165    0
8     Gray  128  128  128
9      Red  255    0    0
10   Black    0    0    0
11   Green    0  128    0
12    Blue    0    0  255
13  Violet  238  130  238
14   Lilac  200  162  200


Question 16 : Le DataFrame colors avec une seule colonne , qui contient le nom de chaque combinaison unique de couleur

In [10]:
#Q16

couleurs_combinees = champignons[champignons['COLOR'].str.count('-') > 0]

combinaisons_uniques = couleurs_combinees['COLOR'].unique()

colors = pd.DataFrame({'Combined_Color': combinaisons_uniques})

print(colors)

   Combined_Color
0    White-Yellow
1     White-Brown
2      Pink-Brown
3    Purple-Brown
4      White-Pale
..            ...
90     Green-Pink
91     Gray-White
92       Red-Pale
93    Gray-Yellow
94     Purple-Red

[95 rows x 1 columns]


Question 17 : compléter colors avec trois colonnes "R" "G" "B"

In [11]:
#Q17

colors['Color1'] = colors['Combined_Color'].str.split('-').str.get(0)
colors['Color2'] = colors['Combined_Color'].str.split('-').str.get(1)

merge_1 = colors.merge(colors_df, left_on='Color1', right_on='Color', suffixes=('', '_1'))
merge_2 = merge_1.merge(colors_df, left_on='Color2', right_on='Color', suffixes=('', '_2'))

merge_2['R_mean'] = merge_2[['R', 'R_2']].mean(axis=1)
merge_2['G_mean'] = merge_2[['G', 'G_2']].mean(axis=1)
merge_2['B_mean'] = merge_2[['B', 'B_2']].mean(axis=1)

colors = merge_2[['Combined_Color', 'R_mean', 'G_mean', 'B_mean']]

print(colors)

   Combined_Color  R_mean  G_mean  B_mean
0    White-Yellow   255.0   255.0   127.5
1     Pink-Yellow   255.0   223.5   101.5
2     Pale-Yellow   247.5   238.0   107.5
3    Brown-Yellow   210.0   148.5    21.0
4     Gray-Yellow   191.5   191.5    64.0
..            ...     ...     ...     ...
90    Brown-Green    82.5    85.0    21.0
91     Gray-Green    64.0   128.0    64.0
92   Orange-Green   127.5   146.5     0.0
93   Yellow-Green   127.5   191.5     0.0
94     Blue-Green     0.0    64.0   127.5

[95 rows x 4 columns]


Question 18 : Compléter champignons avec trois couleur , supression de la colonne Color , donner des valeur pour les NA

In [12]:
#Q18

champignons = champignons.merge(colors[['Combined_Color', 'R_mean', 'G_mean', 'B_mean']], how='left', left_on='COLOR', right_on='Combined_Color')

champignons.rename(columns={'R_mean': 'R', 'G_mean': 'G', 'B_mean': 'B'}, inplace=True)

champignons.drop(columns=['COLOR', 'Combined_Color'], inplace=True)

champignons[['R', 'G', 'B']] = champignons[['R', 'G', 'B']].fillna(-255)

print(champignons)


      TYPE  SHAPE_Polypore  SHAPE_Convex  SHAPE_Bellshaped  SHAPE_Depressed  \
0      1.0               1             0                 0                0   
1      0.0               0             1                 0                0   
2      0.0               0             1                 0                0   
3      0.0               0             0                 1                0   
4      0.0               0             1                 0                0   
...    ...             ...           ...               ...              ...   
1153   1.0               0             0                 0                0   
1154   1.0               0             0                 0                0   
1155   1.0               0             0                 0                0   
1156   1.0               0             0                 0                0   
1157   0.0               0             1                 0                0   

      SHAPE_CupFungi  SHAPE_CoralFungi  SHAPE_Conic

In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

Guestion 19 : Séparation de données en un jeu d'entrainement et de test.

In [14]:
#Q19

X = champignons.drop('TYPE', axis=1) 
y = champignons['TYPE'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

print("Taille de l'ensemble d'entraînement :", len(X_train))
print("Taille de l'ensemble de test :", len(X_test))

Taille de l'ensemble d'entraînement : 868
Taille de l'ensemble de test : 290


In [15]:
#Question 20 : Entrainer le model SVC

In [16]:
#Q20

model = SVC(random_state=1)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Score de précision: {accuracy}")

conf_matrice = confusion_matrix(y_test, y_pred)
print("Matrice de confusion:")
print(conf_matrice)

Score de précision: 0.4413793103448276
Matrice de confusion:
[[  0   1   0   0]
 [  0 128   0   0]
 [  0  49   0   0]
 [  0 112   0   0]]


In [17]:
#Question 21 : reprendre la question 20 en standardisant les colonnes dans un premier temps.

In [18]:
#Q21

scaler = StandardScaler()
X_train_echelle = scaler.fit_transform(X_train)
X_test_echelle = scaler.transform(X_test)

model = SVC(random_state=1)
model.fit(X_train_echelle, y_train)

y_pred = model.predict(X_test_echelle)

accuracy = accuracy_score(y_test, y_pred)
print(f"Score de précision: {accuracy}")

conf_matrice = confusion_matrix(y_test, y_pred)
print("Matrice de confusion:")
print(conf_matrice)

Score de précision: 0.5172413793103449
Matrice de confusion:
[[  1   0   0   0]
 [  0 102   1  25]
 [  0  28   1  20]
 [  0  60   6  46]]


Question 22 : arbre de décision

In [19]:
#Q22

tree_model = DecisionTreeClassifier(max_depth=3, random_state=1)
tree_model.fit(X_train, y_train)

y_pred_tree = tree_model.predict(X_test)

accuracy_tree = accuracy_score(y_test, y_pred_tree)
print(f"Score précision (non standarisée): {accuracy_tree}")

conf_matrice_arbre = confusion_matrix(y_test, y_pred_tree)
print("Matrice de confusion (données non standarisée):")
print(conf_matrice_arbre)

tree_model.fit(X_train_echelle, y_train)

y_pred_tree_echelle = tree_model.predict(X_test_echelle)

accuracy_tree_echelle = accuracy_score(y_test, y_pred_tree_echelle)
print(f"Score précision (non standarisée): {accuracy_tree_echelle}")

conf_matrice_arbre_echelle = confusion_matrix(y_test, y_pred_tree_echelle)
print("Matrice de confusion (donées standarisée):")
print(conf_matrice_arbre_echelle)

Score précision (non standarisée): 0.49310344827586206
Matrice de confusion (données non standarisée):
[[  0   1   0   0]
 [  0 124   0   4]
 [  0  40   0   9]
 [  0  92   1  19]]
Score précision (non standarisée): 0.49310344827586206
Matrice de confusion (donées standarisée):
[[  0   1   0   0]
 [  0 124   0   4]
 [  0  40   0   9]
 [  0  92   1  19]]


Question 23 : la représentation graphique de l'arbre de décision

In [20]:
#Q23

from sklearn.tree import export_graphviz
import graphviz

feature_names = champignons.columns.drop('TYPE')

export_graphviz(tree_model, out_file="arbre_decision.dot", 
                feature_names=feature_names, 
                filled=True)

with open("arbre_decision.dot") as f:
    dot_graph = f.read()
graphviz.Source(dot_graph).view()

'Source.gv.pdf'

Question 24 : sauvegarder chacun des deux modèles dans un fichier

In [21]:
#Q24

from joblib import dump

dump(tree_model, 'arbre_decision.joblib')

['arbre_decision.joblib']